In [78]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import logging
%matplotlib inline

In [79]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, roi_vertices, color=[255, 0, 0], thickness=6):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    lefts = []
    left_slopes = []
    rights = []
    right_slopes = []
    bounds = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            #cv2.line(img,(x1,y1),(x2,y2),color,thickness)
            if(is_roi_vertex([x1,y1],roi_vertices) or is_roi_vertex([x2,y2],roi_vertices) or (x2-x1<1e-4)):
                bounds.append(line)
                continue
            else:
                m = (y2-y1)/(x2-x1)
                if(m>0):
                    lefts.append(line)
                    #cv2.line(img,(x1,y1),(x2,y2),color,thickness)
                else:
                    rights.append(line)
                    #cv2.line(img,(x1,y1),(x2,y2),color,thickness)
    y = [v[1] for v in roi_vertices]
    if(len(rights)>0):
        right_seg = get_projection(rights,max(y),min(y)+70)
        logging.info("Right segment: {}".format(right_seg))
        if(not right_seg==None):
            cv2.line(img,(right_seg[0],right_seg[1]),(right_seg[2],right_seg[3]),[0,0,255],thickness)
        else:
            logging.debug("Right seg: nil")
    if(len(lefts)>0):
        left_seg = get_projection(lefts,max(y),min(y)+70)
        if(not left_seg==None):
            cv2.line(img,(left_seg[0],left_seg[1]),(left_seg[2],left_seg[3]),[0,255,0],thickness)
        else:
            logging.debug("Left seg: nil")
            
def is_roi_vertex(p,roi_vertices):
    for v in roi_vertices:
        if(np.linalg.norm(p-v)<30):
            return True
    return False

def get_projection(segments,bottom,top):
    try:
        slopes = []
        longest = []
        max_length = 0
        eps=1e-2
        for seg in segments:
            for x1,y1,x2,y2 in seg:
                n = np.linalg.norm([x2-x1,y2-y1])
                if(x2-x1<1e-4):
                    continue
                m = (y2-y1)/(x2-x1)
                if(math.fabs(m)>eps):
                    if(n>max_length):
                        longest = seg[0]
                        max_length = n
                    slopes.append(m)
        right_slopes = []
        logging.debug(slopes)
        std = np.asarray(slopes).std()
        mean = np.asarray(slopes).mean()
        for m in slopes:
            logging.debug("Check: {}, mean={}, {} < {}".format(m,mean,math.fabs(m-mean),std))
            if(math.fabs(m-mean)<=std):
                right_slopes.append(m)
        logging.debug(right_slopes)
        if(len(right_slopes)==0):
            logging.info("nothing in right_slopes")
            return None
        avg_slope = np.mean(np.asarray(right_slopes))
        if(math.fabs(avg_slope)<eps):
            logging.info("avg_slope is too small : {}".format(avg_slope))
            return None
        x1,y1,x2,y2=longest[0],longest[1],longest[2],longest[3]
        intercept = y1-(avg_slope)*x1
        bottom_x = int((bottom-intercept)/avg_slope)
        top_x = int((top-intercept)/avg_slope)
        return [bottom_x,bottom,top_x,top]
    except:
        logging.info("Exception in get_projection: {}".format(traceback.format_exc()))
        return None
        
def hough_lines(img, roi_vertices, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines,roi_vertices)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [100]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2

def get_bounds(image):
    height = int(image.shape[0])
    width = int(image.shape[1])
    vertices = np.array([[50,height-2],[width-50,height-2],[width/2,height/2]])
    return vertices.astype(int)

def get_edges(image,vertices):
    roi = region_of_interest(image,[vertices])
    # draw greyscale
    gray = cv2.cvtColor(roi, cv2.COLOR_RGB2GRAY)
    
    # get edges
    kernel_size = 3
    blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size), 0)
    low_threshold = 60
    high_threshold = 180
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)
    return edges

def get_hough_lines(edges,vertices,rho, theta, threshold, min_line_length, max_line_gap):
    line_image = np.copy(edges)*0 # creating a blank to draw lines on
    line_image = hough_lines(edges, vertices, rho, theta, threshold, min_line_length, max_line_gap)
    return line_image

def draw_lanes(image,rho=1e-1, theta=np.pi/(180*8), threshold=10, min_line_length=100, max_line_gap=200):
    bounds = get_bounds(image)
    edges = get_edges(image,bounds)
    line_image = np.copy(edges)*0 # creating a blank to draw lines on
    line_image = hough_lines(edges, bounds, rho, theta, threshold, min_line_length, max_line_gap)
    i = weighted_img(image,line_image)
    return i



In [101]:
import glob
import os
logger = logging.getLogger()
logger.setLevel(logging.WARN)
import warnings
warnings.filterwarnings("ignore")

def search_params(fs):
    for cnt,image_f in enumerate(fs):
        rho, theta, threshold, min_line_length, max_line_gap=.1,np.pi/(180*8), \
        np.linspace(10,20,2).astype(int),\
        np.linspace(160,480,1).astype(int),\
        np.linspace(200,400,1).astype(int) 
        for th in threshold:
            for mll in min_line_length:
                for mlg in max_line_gap:
                    s = "{}: th:{}\tmll:{}\tmlg:{}".format(cnt, th,mll,mlg)
                    logging.info("{}=============={}={}===========".format(cnt,os.path.abspath(image_f),s))
                    i=draw_lanes(mpimg.imread(image_f), rho,theta,th,mll,mlg)
                    plt.figure()  
                    plt.annotate(s, xy=(36,36))
                    plt.imshow(i)
def lanes(fs):
    for cnt,image_f in enumerate(fs):
        rho, theta, threshold, min_line_length, max_line_gap=.1,np.pi/(180*8),12,100,160
        i=draw_lanes(mpimg.imread(image_f), rho, theta, threshold, min_line_length, max_line_gap)
        s = "{}:th:{}\tmll:{}\tmlg:{}".format(cnt,threshold, min_line_length, max_line_gap)
        plt.figure()  
        plt.annotate(s, xy=(36,36))
        plt.imshow(i)

                    

In [113]:
fs = glob.glob('challenge/*.jpg')
lanes([])


In [104]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [114]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    rho, theta, threshold, min_line_length, max_line_gap=.1,np.pi/(180*8),10,100,100
    i=draw_lanes(image, rho, theta, threshold, min_line_length, max_line_gap)
    # you should return the final output (image with lines are drawn on lanes)

    return i

In [ ]:
for v in ["solidWhiteRight.mp4", "solidYellowLeft.mp4", "challenge.mp4"]:
    clip = VideoFileClip(v)
    outname = os.path.basename(v).split('.')[0]+"_ann.mp4"
    outclip = clip.fl_image(process_image)
    outclip.write_videofile(outname,audio=False)
    

[MoviePy] >>>> Building video solidWhiteRight_ann.mp4
[MoviePy] Writing video solidWhiteRight_ann.mp4



100%|█████████▉| 221/222 [01:04<00:00,  3.46it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: solidWhiteRight_ann.mp4 

[MoviePy] >>>> Building video solidYellowLeft_ann.mp4
[MoviePy] Writing video solidYellowLeft_ann.mp4



 27%|██▋       | 187/682 [00:52<02:34,  3.20it/s]

In [90]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|█████████▉| 221/222 [01:03<00:00,  3.34it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 47.5 s, sys: 14.8 s, total: 1min 2s
Wall time: 1min 4s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [8]:
white_output = 'white.mp4'
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [91]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|█████████▉| 681/682 [03:18<00:00,  3.62it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

CPU times: user 2min 27s, sys: 45.5 s, total: 3min 13s
Wall time: 3min 19s


In [165]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


The current pipeline first gathers left side , and right side hough line segments. It then determines a projection for each side which corresponds to a continuous line for a lane marking. 

I imagine a few situations where this algorithm may encounter problems:
    1. situations where there are other line markers closeby. eg: parking line markers, or bike lane markers
    2. curves in roads - when linemarkers are not straight lines and are windy curves
    3. Stretches of roads with faded lane markers
    4. Stretches of roads where there may be trees,  and other vertical objects that may contribute false edges
    5. Stretches of roads with strong shadows that may contribute false edges
    
In addition, the processing rate seems to be about 3 frames per second which may be insufficient for real world conditions.
    

## Submission

If you're satisfied with your video outputs it's time to submit!


## Completed Optional Challenge



In [92]:

challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


100%|██████████| 251/251 [01:51<00:00,  2.43it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

CPU times: user 1min 26s, sys: 22.3 s, total: 1min 48s
Wall time: 1min 52s


In [16]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))